In [7]:
import pandas as pd
from dfply import *

import sys
from sshtunnel import SSHTunnelForwarder
import logging
sys.path.append('/Users/sdc/Documents/bida_lab/db/P1_extubationfailure/서대철')

import src.data_extraction.access_database as db   # DB 연결 관련 모듈
import src.utils.utils as utils   # 기타 유틸리티 모듈

## Features: Severity (GCS)

### 1. Subjectlist 가져오기

In [2]:
# 데이터 저장위치
# data_dir = './data'   # 수정 쌤 디렉토리에서 사용
data_dir = '../../outputs'   # 내 디렉에서 사용
sub_list_dir = data_dir + '/' + 'sub1list.csv'


subjectlist1 = pd.read_csv(sub_list_dir, parse_dates=['admittime', 'dischtime'])
sub1list = subjectlist1 >> select('subject_id','hadm_id','stay_id', 'admittime', 
                                'dischtime', 'gender', 'age', 'height', 
                                'weight')
sub1list = sub1list.drop_duplicates(subset='stay_id')   # remove dup
sub1list

,subject_id,hadm_id,stay_id,admittime,dischtime,gender,age,height,weight
0,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0
1,10003400,23559586,38383343,2137-08-04 00:07:00,2137-09-02 17:05:00,F,75.589414,165.0,99.6
2,10004401,27939719,31202136,2144-04-11 03:31:00,2144-04-13 17:31:00,M,85.274942,NaN,80.4
3,10004401,29988601,32773003,2144-01-23 07:58:00,2144-02-06 11:45:00,M,85.059155,168.0,76.0
4,10005817,28661809,31316840,2135-01-03 21:54:00,2135-01-19 18:36:00,M,69.008724,175.0,88.8
...,...,...,...,...,...,...,...,...,...
7226,19995595,21784060,34670930,2126-10-17 21:43:00,2126-11-12 17:35:00,M,68.793734,163.0,94.7
7227,19997367,20617667,35616526,2126-04-20 07:15:00,2126-05-19 14:15:00,F,63.299259,147.0,59.0
7228,19999068,21606769,30143796,2161-08-24 04:10:00,2161-09-02 19:00:00,M,63.643884,170.0,55.8
7229,19999442,26785317,32336619,2148-11-19 10:00:00,2148-12-04 16:25:00,M,43.884161,193.0,107.5


### 2. DB 연결 후 필요한 테이블 가져오기
- gcs


In [5]:
# 1. DB 파라미터 설정

# 데이터 저장위치
# output_dir = './data'   # 수정 쌤 디렉토리에서 사용
output_dir = '../../outputs'   # 내 디렉에서 사용


# SSH 연결 설정
ssh_config = {
    'ssh_ip': '210.91.223.248',
    'ssh_port': 35430,
    'ssh_username': "medicalai",
    'ssh_password': "medicalai731!",
    'remote_bind_address': ('localhost', 35432)  # 데이터베이스 호스트 및 포트
}

# 데이터베이스 연결 설정
db_config = {
    'database': 'mimiciv',
    'user': 'mai_admin',
    'password': 'admin1q2w!@',
    'host': 'localhost',  # SSH 터널을 사용하면, 이 값은 'localhost'가 됩니다
    'port': 35432  # 이 값은 SSH 터널 로컬 포트 바인딩에 의해 대체됩니다
}


# 테이블 쿼리 설정
tables_query = {
    'gcs': "select * from mimiciv_derived.gcs;"
                }

db.print_config_info(db_config, tables_query)

--------- Database Configuration ---------
Database Name: mimiciv
User: mai_admin
Password: ***********
Host: localhost
Port: 35432

--------- SQL Queries for Required Tables ---------
gcs: select * from mimiciv_derived.gcs;



In [6]:
# 2. db 접속, 테이블 추출, 'dataframes' 라는 딕셔너리에 테이블 저장

try:
    conn, tunnel = db.connect_to_database_via_ssh(db_config, ssh_config)
    if conn is not None:
        # 데이터베이스 작업 수행
        dataframes = db.retrieve_data(conn, tables_query)
    else:
        logging.error("Failed to connect to the database.")
except Exception as e:
    logging.error(f"An error occurred: {e}")
finally:
    if conn is not None:
        db.disconnect_database(conn)


# 데이터프레임 변수로 저장
## Severity: GCS
gcs = dataframes['gcs']


INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_8.2p1)
INFO:paramiko.transport:Authentication (password) successful!
INFO:root:SSH TUNNEL ESTABLISHED...
INFO:root:mimiciv DATABASE CONNECTED VIA SSH.
INFO:root:Retrieved gcs: (1637763, 8)
INFO:root:DATABASE CONNCETION CLOSED.


In [8]:
# datetime 변환
time_cols = ['charttime']
gcs = utils.to_datetime(gcs, time_cols)
gcs

Converted charttime to datetime.


,subject_id,stay_id,charttime,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable
0,12466550,30000153,2174-09-29 12:45:00,15.0,5.0,0.0,3.0,1
1,12466550,30000153,2174-09-29 16:26:00,15.0,6.0,0.0,4.0,1
2,12466550,30000153,2174-09-29 17:37:00,15.0,6.0,0.0,4.0,1
3,12466550,30000153,2174-09-29 18:00:00,9.0,5.0,1.0,3.0,0
4,12466550,30000153,2174-09-29 19:00:00,9.0,5.0,1.0,3.0,0
...,...,...,...,...,...,...,...,...
1637758,17840864,39999810,2115-12-05 00:00:00,14.0,6.0,5.0,3.0,0
1637759,17840864,39999810,2115-12-05 04:00:00,14.0,6.0,5.0,3.0,0
1637760,17840864,39999810,2115-12-05 08:28:00,15.0,6.0,5.0,4.0,0
1637761,17840864,39999810,2115-12-05 12:00:00,15.0,6.0,5.0,4.0,0


### 3. 테이블 결합하기

In [9]:
# gcs (Glasgow Coma Score) 결합 (stay_id 기준)
sub1list_gcs = pd.merge(sub1list, gcs[['subject_id', 'stay_id', 'charttime', 'gcs', 'gcs_verbal', 'gcs_eyes', 'gcs_unable']], on=['subject_id', 'stay_id'], how='left')
sub1list_gcs

,subject_id,hadm_id,stay_id,admittime,dischtime,gender,age,height,weight,charttime,gcs,gcs_verbal,gcs_eyes,gcs_unable
0,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 08:00:00,15.0,0.0,1.0,1.0
1,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 10:00:00,15.0,0.0,2.0,1.0
2,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 12:00:00,15.0,0.0,3.0,1.0
3,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 14:00:00,15.0,0.0,3.0,1.0
4,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 16:00:00,15.0,0.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380738,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-17 13:42:00,M,58.694264,178.0,77.5,2164-09-16 20:21:00,15.0,0.0,1.0,1.0
380739,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-17 13:42:00,M,58.694264,178.0,77.5,2164-09-17 00:00:00,15.0,0.0,1.0,1.0
380740,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-17 13:42:00,M,58.694264,178.0,77.5,2164-09-17 04:00:00,15.0,0.0,1.0,1.0
380741,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-17 13:42:00,M,58.694264,178.0,77.5,2164-09-17 08:08:00,15.0,0.0,1.0,1.0


### 4. Null 값 처리, 중복치 확인

In [25]:
# null 값 확인
print(f'null count: {sub1list_gcs.gcs.isna().sum()}')
sub1list_gcs[sub1list_gcs.gcs.isna()]   # null 값 케이스 12건 확인. charttime도 없기 때문에 제거하겠음.

sub1list_gcs_notna = sub1list_gcs[sub1list_gcs.gcs.notna()] 

null count: 12


In [26]:
# 중복치 확인 -> 없음.
## 중복치 기준: stay_id 내에서 charttime이 동일한 경우

duplicates = sub1list_gcs_notna[sub1list_gcs_notna.duplicated(subset=['stay_id', 'charttime'], keep=False)]
duplicates_sorted = duplicates.sort_values(by=['stay_id', 'charttime'])

duplicates_sorted


,subject_id,hadm_id,stay_id,admittime,dischtime,gender,age,height,weight,charttime,gcs,gcs_verbal,gcs_eyes,gcs_unable


### 5. hadm_id, stay_id 별로 시간순 (charttime) 정렬

In [27]:
sub1list_gcs_notna.sort_values(by=['subject_id', 'hadm_id', 'stay_id', 'charttime'], inplace=True)
sub1list_gcs_notna

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub1list_gcs_notna.sort_values(by=['subject_id', 'hadm_id', 'stay_id', 'charttime'], inplace=True)



,subject_id,hadm_id,stay_id,admittime,dischtime,gender,age,height,weight,charttime,gcs,gcs_verbal,gcs_eyes,gcs_unable
0,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 08:00:00,15.0,0.0,1.0,1.0
1,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 10:00:00,15.0,0.0,2.0,1.0
2,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 12:00:00,15.0,0.0,3.0,1.0
3,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 14:00:00,15.0,0.0,3.0,1.0
4,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-20 05:15:00,F,77.018296,157.0,65.0,2131-01-11 16:00:00,15.0,0.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380738,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-17 13:42:00,M,58.694264,178.0,77.5,2164-09-16 20:21:00,15.0,0.0,1.0,1.0
380739,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-17 13:42:00,M,58.694264,178.0,77.5,2164-09-17 00:00:00,15.0,0.0,1.0,1.0
380740,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-17 13:42:00,M,58.694264,178.0,77.5,2164-09-17 04:00:00,15.0,0.0,1.0,1.0
380741,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-17 13:42:00,M,58.694264,178.0,77.5,2164-09-17 08:08:00,15.0,0.0,1.0,1.0


### 6. 데이터 저장



In [28]:
file_name = data_dir + '/' + 'sub1list_gcs.csv'
sub1list_gcs_notna.to_csv(file_name, index=False, encoding='utf-8-sig')